In [5]:
import os
import json
from shapely.geometry import Polygon
import sys
import numpy as np
import pdal
import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

In [6]:
import pdal
# loading json file
def read_json(json_path):
    try:
        with open(json_path) as js:
            json_obj = json.load(js)
        return json_obj

    except FileNotFoundError:
        print('File not found.')



In [7]:
a = read_json("untitled.json")

In [8]:
a

{'pipeline': [{'bounds': '([-10425171.940, -10425161.940], [5164494.710, 5164500.710])',
   'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json',
   'type': 'readers.ept',
   'tag': 'readdata'},
  {'limits': 'Classification![7:7]',
   'type': 'filters.range',
   'tag': 'nonoise'},
  {'in_srs': 'EPSG:3857',
   'out_srs': 'EPSG:4326',
   'tag': 'reprojectUTM',
   'type': 'filters.reprojection'},
  {'filename': 'iowa.csv', 'tag': 'writerslas', 'type': 'writers.text'}]}

In [9]:
pipe=pdal.Pipeline(json.dumps(a))
pipe.execute()

21

In [10]:
MINX, MINY, MAXX, MAXY = [-93.759055, 41.925015, -93.766155, 41.935015]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

In [11]:
class LidarProcessor:
    """
    This class contains functons useful for fetching, manipulating, and visualizing LIDAR point cloud data.
    """

    def __init__(self, public_data_url: str = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json", pipeline_json_path: str="untitled.json") -> None:
            
        self.pipeline_json = read_json(pipeline_json_path)
        self.public_data_url = public_data_url
        self.input_epsg = 3857
    
        
    def get_polygon_boundaries(polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=4326, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    
    def read_csv(csv_path, missing_values=["n/a", "na", "undefined"]):
        try:
            df = pd.read_csv(csv_path, na_values=missing_values)
            return df

        except FileNotFoundError:
            print('File not found.')
            


In [12]:
df = LidarProcessor.read_csv("iowa.csv")
df

,Classification,EdgeOfFlightLine,GpsTime,Intensity,NumberOfReturns,OriginId,PointSourceId,ReturnNumber,ScanAngleRank,ScanDirectionFlag,UserData,X,Y,Z
0,1.0,0.0,56180.805,44.0,1.0,17583.0,85.0,1.0,8.0,1.0,69.0,-93.651,42.023,307.86
1,1.0,0.0,56180.805,77.0,1.0,17583.0,85.0,1.0,8.0,1.0,69.0,-93.651,42.023,308.66
2,1.0,0.0,56180.832,81.0,1.0,17583.0,85.0,1.0,8.0,1.0,69.0,-93.651,42.023,302.85
3,1.0,0.0,56180.788,86.0,1.0,17583.0,85.0,1.0,8.0,0.0,69.0,-93.651,42.023,308.87
4,1.0,0.0,56180.815,54.0,1.0,17583.0,85.0,1.0,8.0,0.0,69.0,-93.651,42.023,307.76
5,1.0,0.0,56180.815,53.0,1.0,17583.0,85.0,1.0,8.0,0.0,69.0,-93.651,42.023,307.74
6,1.0,0.0,56180.815,80.0,1.0,17583.0,85.0,1.0,8.0,0.0,69.0,-93.651,42.023,305.62
7,1.0,0.0,56180.832,79.0,1.0,17583.0,85.0,1.0,8.0,1.0,69.0,-93.651,42.023,304.27
8,1.0,0.0,56180.832,53.0,1.0,17583.0,85.0,1.0,8.0,1.0,69.0,-93.651,42.023,304.16
9,1.0,0.0,56180.843,52.0,2.0,17583.0,85.0,1.0,8.0,0.0,69.0,-93.651,42.023,301.01
